In [1]:
# Importing dependencies
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from utils import *

Falling back to using features:  <ORB 0000025E74AA4F90>
For BoVW, use feature:  <ORB 0000025E74AA4F90>


In [2]:
# Check OpenCV version, should be >= 3.3.0
print(cv2.__version__)

4.5.2


In [3]:
# Loading images -> neg, pos
paths = [params.DATA_TRAINING_PATH_NEG, params.DATA_TRAINING_PATH_POS]

# Build a list of class_names
class_names = [get_class_name(class_number) for class_number in range(len(params.DATA_CLASS_NAMES))]

# Specify number of subwindow samples to take
# Use 0 to take the whole image
sampling_sizes = [0, 0]

# Take samples from center
sample_from_center = [True, True]

# Perform image loading
start_load = cv.getTickCount()
imgs_data = load_images(paths, 
                        class_names, 
                        sampling_sizes, 
                        sample_from_center, 
                        params.DATA_WINDOW_OFFSET_FOR_TRAINING_SAMPLES, 
                        params.DATA_WINDOW_SIZE)

print(('Loaded {} images'.format(len(imgs_data))))
print_duration(start_load)

Loaded 10000 images
Took 10.2s


In [4]:
# Perform Bag of Visual Words feature construction

print('\nComputing descriptors ...')
start_describe = cv.getTickCount()
[img_data.compute_bovw_descriptor() for img_data in tqdm(imgs_data)]
print_duration(start_describe)

print('\nClustering ...')
start_cluster = cv.getTickCount()
dictionary = generate_dictionary(imgs_data, params.BOVW_DICTIONARY_SIZE)
print_duration(start_cluster)

print('\nGenerating histograms ...')
start_hist = cv.getTickCount()
[img_data.generate_bovw_hist(dictionary) for img_data in tqdm(imgs_data)]
print_duration(start_hist)

  0%|          | 0/10000 [00:00<?, ?it/s]
Computing descriptors ...
100%|██████████| 10000/10000 [00:20<00:00, 485.72it/s]
Took 20.6s

Clustering ...
  0%|          | 32/10000 [00:00<00:31, 316.84it/s]Took 12m:25.5s

Generating histograms ...
100%|██████████| 10000/10000 [00:32<00:00, 309.61it/s]Took 32.3s



In [5]:
# Train an SVM based on features
start_svm = cv.getTickCount()

svm = cv.ml.SVM_create()
svm.setType(cv.ml.SVM_C_SVC)
svm.setKernel(params.BOVW_SVM_KERNEL)

# Compile samples for each training image
samples = get_bovw_histograms(imgs_data)

# Get class label for each training image
class_labels = get_class_labels(imgs_data)

# Specify term criteria for training
svm.setTermCriteria((cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_COUNT, params.BOVW_SVM_MAX_TRAINING_ITERATIONS, 1.e-06))

# Perform autotraining
svm.trainAuto(samples, cv.ml.ROW_SAMPLE, class_labels, kFold=10, balanced=True)

# Save trained SVM
svm.save(params.BOVW_SVM_PATH)
print_duration(start_svm)

Took 36m:52.1s


In [7]:
# Performance measure of the SVM
output = svm.predict(samples)[1].ravel()
error = (np.absolute(class_labels.ravel() - output).sum()) / float(output.shape[0])

if error < (1.0 / len(params.DATA_CLASS_NAMES)):
    print('Trained SVM obtained {}% error in train dataset'.format(round(error*100, 2)))
    print('-- and the accuracy is {}%.'.format(round((1.0 - error)*100,2)))
else:
    print('Failed to train SVM. {}% error'.format(round(1.0 - error)*100, 2))

Trained SVM obtained 6.88% error in train dataset
-- and the accuracy is 93.12%.


In [9]:
# Loading testing data
DATA_TESTING_PATH_NEG = '.\\dataset\\Test\\WithoutMask'
DATA_TESTING_PATH_POS = '.\\dataset\\Test\\WithMask'

test_paths = [DATA_TESTING_PATH_NEG, DATA_TESTING_PATH_POS]

# Perform image loading
start_load = cv.getTickCount()
imgs_test = load_images(test_paths, 
                        class_names, 
                        sampling_sizes, 
                        sample_from_center, 
                        params.DATA_WINDOW_OFFSET_FOR_TRAINING_SAMPLES, 
                        params.DATA_WINDOW_SIZE)

print(('Loaded {} images'.format(len(imgs_test))))
print_duration(start_load)

Loaded 992 images
Took 0.8s


In [10]:
# Perform feature extraction
print('\nComputing descriptors ...')
start_describe = cv.getTickCount()
[img_test.compute_bovw_descriptor() for img_test in tqdm(imgs_test)]
print_duration(start_describe)

# Genenrating histogram using generated dictionary
print('\nGenerating histograms ...')
start_hist = cv.getTickCount()
[img_test.generate_bovw_hist(dictionary) for img_test in tqdm(imgs_test)]
print_duration(start_hist)

 20%|██        | 203/992 [00:00<00:00, 822.04it/s]
Computing descriptors ...
  5%|▌         | 52/992 [00:00<00:01, 509.86it/s]Took 1.5s

Generating histograms ...
100%|██████████| 992/992 [00:02<00:00, 393.18it/s]Took 2.5s



In [13]:
# Compile samples for each training image
test = get_bovw_histograms(imgs_test)
# Get class label for each training image
class_labels = get_class_labels(imgs_test)

# Performance measure of the SVM on new data
output_test = svm.predict(test)[1].ravel()
error_test = (np.absolute(class_labels.ravel() - output_test).sum()) / float(output_test.shape[0])

if error_test < (1.0 / len(params.DATA_CLASS_NAMES)):
    print('Trained SVM obtained {}% error in train dataset'.format(round(error_test*100, 2)))
    print('-- and the accuracy is {}%.'.format(round((1.0 - error_test)*100,2)))
else:
    print('Failed to train SVM. {}% error'.format(round(1.0 - error_test)*100, 2))

Trained SVM obtained 9.27% error in train dataset
-- and the accuracy is 90.73%.
